# Solving RBC Model with Dolo.jl

This worksheet demonstrates how to solve the RBC model with the [dolo](http://econforge.github.io/dolo/) library 
and how to generates impulse responses and stochastic simulations from the solution.

- This notebook is modified from `rbc_model` example distributed with dolo in : ``examples\notebooks\``.
- The model file(.yaml file) is in : ``examples\global_models\``


# YAML file for model specification

Dolo uses [YAML](http://www.yaml.org/spec/1.2/spec.html#Introduction) file which we can read locally or pull off the web. 

The YAML file works similarly to `.mod` file in Dynare, which does a configuration of the model. It includes the following:

1. In `symbols` section, specify different kinds of variables in the model:
    - `parameters:` deep parameters of the model, exp: $\beta$
    - `exogenous:` exogenous shocks of the model, exp: $\epsilon$ (iid shock on productivity)
    - `states:` state variables of the model, exp: $k$ (capital stock)
    - `controls:` control/choice variables of the model, exp: $n$ (labor)
    - `values:` value function in terms of Bellman Equation; `rewards:` utility function.

2. "auxiliary variables": these are variables that can be expressed as a closed form of the formerly-defined variables. Defining these variables will help to simplify the system of equations and sometimes these variables have particular economic meaning. These variables are defined in `definitions`. Once defined, one could use them in model equations.

3. Model section, specified using command `equations:`, consist of:

   Essential parts:
    
    - `arbitrage:` Euler Equation, notice the EE specification is usually accompanied by complementarity condition included in the `arbitrage` configuration.

    - `transition:` Law of motion, this should include both things like capital accumulation and process of shocks.

    Aditionals:

    - `expectations:` A function of purely expected future values.

    - `felicity:` defining utility function; `values:` defining value function;

    - `direct response:` additional equations that may speed up computation.


For more options in model specification please refer to http://www.econforge.org/Dolo.jl/latest/model_specification.html

# Example of RBC

In [1]:
using Dolo
filename="rbc_dtcc_iid.yaml"
;cat $filename

`yaml_import(filename)` reads the YAML file and generates a model object. 

In [29]:
model = yaml_import(filename)

name,Real Business Cycle
filename,rbc_dtcc_iid.yaml
Type,Equation
expectation,\[m_{t} = \frac{\beta}{\left(c_{t+1}\right)^{\sigma}} \left(1-\delta\right)+rk_{t+1}\]
value,\[V_{t} = \left(\frac{\left(c_{t}\right)^{\left(1-\sigma\right)}}{\left(1-\sigma\right)}-\frac{\chi \left(n_{t}\right)^{1+\eta}}{1+\eta}\right)+\beta V_{t+1}\]
transition,"\[z_{t} = \rho z_{t-1}+e_{z,t}\]"
,\[k_{t} = \left(1-\delta\right) k_{t-1}+i_{t-1}\]
direct_response,\[n_{t} = \left(\frac{\left(1-\alpha\right) \text{exp}\left(z_{t}\right) \left(k_{t}\right)^{\alpha} m_{t}}{\chi}\right)^{\frac{1}{\eta+\alpha}}\]
,\[i_{t} = \left(\text{exp}\left(z_{t}\right) \left(k_{t}\right)^{\alpha} \left(n_{t}\right)^{\left(1-\alpha\right)}-\left(m_{t}\right)^{\frac{-1}{\sigma}}\right)\]
felicity,\[u_{t} = \left(\frac{\left(c_{t}\right)^{\left(1-\sigma\right)}}{\left(1-\sigma\right)}-\frac{\chi \left(n_{t}\right)^{1+\eta}}{1+\eta}\right)\]
arbitrage,\[\left(\chi \left(n_{t}\right)^{\eta} \left(c_{t}\right)^{\sigma}-w_{t}\right)\]


# Steady State values

The model file already has values for steady-state variables stated in the calibration section so we can go ahead and check that they are correct by computing the model equations at the steady state.

In [38]:
model.calibration[:controls]

2-element Array{Float64,1}:
 0.33   
 0.20961

# Solving the model

Dolo offers several [algorithms](http://www.econforge.org/Dolo.jl/latest/algos.html) to solve the model. One is `time_iteration`, and its alternative faster command `improved_time_iteration`.

In [39]:
@time sol_global = improved_time_iteration(model)

------------------------------------------------------------------------------------------------------------------------
N	f_x		d_x	Time_residuals	Time_inversion	Time_search	Lambda_0	N_invert	N_search	
------------------------------------------------------------------------------------------------------------------------
1      9.243360e-01 1.754878e-01     0.0120         0.0390         0.00600        0.000           40               2    
2      8.657377e-01 3.015945e-02     0.0130         0.0240         0.00400        0.000           62               1    
3      2.466747e-01 0.000000e+00     0.0120         0.0460         0.00300        0.000           102              1    
4      2.654260e-02 4.962946e-03     0.0100         0.0260         0.00300        0.000           80               1    
5      1.018833e-03 2.982574e-05     0.0170         0.0300         0.00300        0.000           92               1    
6      2.947272e-06 2.230168e-07     0.0100         0.0370         0.003

Results of Improved Time Iteration Algorithm
 * Number of iterations: 7
 * Complementarities: true
 * Decision Rule type: Dolo.CubicDR{Dolo.EmptyGrid,Dolo.CartesianGrid{2},2,2}
 * Convergence: true
 * Contractivity: 0.0
   * |x - x'| < 1.0e-08: true


# Decision rule

... and we get the decision rule and plot optimal investment and labour for different levels of capital.

In [40]:
dr_global = sol_global.dr

tab_global = Dolo.tabulate(model, dr_global, :k)

2-dimensional AxisArray{Float64,2,...} with axes:
    :V, Symbol[:e_z, :z, :k, :n, :i, :w, :rk, :y, :c]
    :k, [3.77298, 3.78145, 3.78992, 3.79839, 3.80686, 3.81533, 3.8238, 3.83227, 3.84074, 3.84921  …  4.5352, 4.54367, 4.55214, 4.56061, 4.56908, 4.57755, 4.58602, 4.59449, 4.60296, 4.61143]
And data, a 9×100 Array{Float64,2}:
 0.0        0.0        0.0        …  0.0        0.0        0.0      
 0.0        0.0        0.0           0.0        0.0        0.0      
 3.77298    3.78145    3.78992       4.59449    4.60296    4.61143  
 0.344002   0.343703   0.343404      0.318158   0.317922   0.317685 
 0.214225   0.214135   0.214046      0.20521    0.205115   0.20502  
 1.47678    1.4783     1.47981    …  1.61712    1.6185     1.61987  
 0.0663179  0.0661798  0.0660421     0.0551551  0.0550597  0.0549645
 0.758232   0.758351   0.758468      0.767907   0.767992   0.768075 
 0.544007   0.544215   0.544422      0.562697   0.562876   0.563055 

In [41]:
using Plots
gr()

figure_1 = plot(
    plot(tab_global[:k], tab_global[:i], label="Global", title = "Investment", ylabel = "i", line = 3),
    plot(tab_global[:k], tab_global[:n], label="Global", title = "Labour", ylabel = "n", line =3)
)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 3.8 
 
 
 4.0 
 
 
 4.2 
 
 
 4.4 
 
 
 4.6 
 
 
 0.206 
 
 
 0.208 
 
 
 0.210 
 
 
 0.212 
 
 
 0.214 
 
 
 Investment 
 
 
 i 
 
<polyline clip-path="url(#clip2902)" style="stroke:#009af9; stroke-width:3; stroke-opacity:1; fill:none" points="
 61.6037,31.4961 63.8129,34.844 66.0221,38.1974 68.2313,41.5616 70.4405,44.942 72.6497,48.344 74.8589,51.7719 77.0681,55.221 79.2773,58.6824 81.4865,62.1473 
 83.6957,65.6068 85.9049,69.0526 88.1142,72.4839 90.3234,75.9063 92.5326,79.3255 94.7418,82.7472 96.951,86.1769 99.1602,89.6171 101.369,93.0651 103.579,96.518 
 105.788,99.9731 107.997,103.427 110.206,106.879 112.415,110.329 114.625,113.778 116.834,117.229 119.043,120.681 121.252,124.137 123.461,127.596 125.671,131.057 
 127.88,134.52 130.089,137.984 132.298,141.45 134.507,144.915 136.717,148.382 138.926,151.85 141.135,155.32 143.344,158.79 145.553,162.263 147.763,165.737 
 149.972,169.212 152.181,172.689 154.39,176.166 156.599,179.645 158.809,183.125 161.018,186.607 163.227,190.089 165.436,193.572 167.646,197.057 169.855,200.543 
 172.064,204.03 174.273,207.518 176.482,211.007 178.692,214.497 180.901,217.988 183.11,221.481 185.319,224.974 187.528,228.468 189.738,231.963 191.947,235.459 
 194.156,238.956 196.365,242.453 198.574,245.952 200.784,249.452 202.993,252.953 205.202,256.454 207.411,259.957 209.62,263.459 211.83,266.963 214.039,270.466 
 216.248,273.97 218.457,277.475 220.666,280.981 222.876,284.489 225.085,287.999 227.294,291.51 229.503,295.021 231.712,298.531 233.922,302.039 236.131,305.545 
 238.34,309.05 240.549,312.556 242.758,316.064 244.968,319.577 247.177,323.096 249.386,326.619 251.595,330.143 253.805,333.664 256.014,337.178 258.223,340.682 
 260.432,344.175 262.641,347.663 264.851,351.152 267.06,354.648 269.269,358.156 271.478,361.679 273.687,365.215 275.897,368.761 278.106,372.313 280.315,375.869 
 
 "/>
 
 
 
 
 Global 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 3.8 
 
 
 4.0 
 
 
 4.2 
 
 
 4.4 
 
 
 4.6 
 
 
 0.320 
 
 
 0.325 
 
 
 0.330 
 
 
 0.335 
 
 
 0.340 
 
 
 Labour 
 
 
 n 
 
<polyline clip-path="url(#clip2903)" style="stroke:#009af9; stroke-width:3; stroke-opacity:1; fill:none" points="
 361.604,31.4961 363.813,35.4134 366.022,39.329 368.231,43.2412 370.441,47.1483 372.65,51.0485 374.859,54.9403 377.068,58.8225 379.277,62.6944 381.487,66.555 
 383.696,70.4036 385.905,74.2394 388.114,78.0625 390.323,81.8739 392.533,85.6743 394.742,89.4647 396.951,93.2461 399.16,97.0186 401.369,100.782 403.579,104.536 
 405.788,108.28 407.997,112.014 410.206,115.737 412.415,119.449 414.625,123.152 416.834,126.844 419.043,130.527 421.252,134.201 423.461,137.865 425.671,141.52 
 427.88,145.165 430.089,148.801 432.298,152.427 434.507,156.044 436.717,159.651 438.926,163.249 441.135,166.838 443.344,170.417 445.553,173.987 447.763,177.548 
 449.972,181.1 452.181,184.643 454.39,188.177 456.599,191.702 458.809,195.218 461.018,198.725 463.227,202.223 465.436,205.713 467.646,209.193 469.855,212.665 
 472.064,216.128 474.273,219.582 476.482,223.028 478.692,226.465 480.901,229.894 483.11,233.314 485.319,236.725 487.528,240.129 489.738,243.523 491.947,246.909 
 494.156,250.287 496.365,253.657 498.574,257.018 500.784,260.371 502.993,263.716 505.202,267.053 507.411,270.382 509.62,273.702 511.83,277.014 514.039,280.319 
 516.248,283.615 518.457,286.903 520.666,290.183 522.876,293.456 525.085,296.721 527.294,299.978 529.503,303.227 531.712,306.468 533.922,309.701 536.131,312.926 
 538.34,316.143 540.549,319.353 542.758,322.555 544.968,325.75 547.177,328.938 549.386,332.119 551.595,335.293 553.805,338.46 556.014,341.617 558.223,344.766 
 560.432,347.906 562.641,351.038 564.851,354.162 567.06,357.278 569.269,360.388 571.478,363.491 573.687,366.59 575.897,369.685 578.106,372.777 580.315,375.869 
 
 "/>
 
 
 
 
 Global

Dolo also offers a convenient way to change parameter values in configuration. Use `set_calibration!(model,para=val)` command one can replace the original parameter value in the model file.

We can use this feature to do comparative stat easily. For example, let's consider a change in the value of $\delta$ in the model:

In [44]:
original_delta=model.calibration.flat[:delta] 

drs = []
delta_values = linspace(0.02, 0.05,5)
for val in delta_values
    print(val)
    set_calibration!(model, delta=val)
    sol = time_iteration(model, verbose=false)
    push!(drs, sol.dr)
end

0.020.02750.0350.04250.05

In [45]:
figure_2 = plot()
for (i,dr) in enumerate(drs)
     sim = Dolo.tabulate(model, dr,:k)
     dv = delta_values[i]
     plot!(sim[:k],sim[:i], label="\\delta=$dv", ylabel = "i", title = "Investment", xlim = [3, 15], ylim = [0.19, 0.26], line = 3)
end

set_calibration!(model,delta=original_delta)

figure_2

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 3 
 
 
 6 
 
 
 9 
 
 
 12 
 
 
 15 
 
 
 0.19 
 
 
 0.20 
 
 
 0.21 
 
 
 0.22 
 
 
 0.23 
 
 
 0.24 
 
 
 0.25 
 
 
 0.26 
 
 
 Investment 
 
 
 i 
 
<polyline clip-path="url(#clip3102)" style="stroke:#009af9; stroke-width:3; stroke-opacity:1; fill:none" points="
 387.229,93.3736 388.27,94.5898 389.31,95.8077 390.351,97.029 391.392,98.2555 392.433,99.4889 393.473,100.731 394.514,101.979 395.555,103.23 396.596,104.48 
 397.637,105.728 398.677,106.968 399.718,108.202 400.759,109.432 401.8,110.659 402.84,111.886 403.881,113.116 404.922,114.349 405.963,115.584 407.004,116.82 
 408.044,118.056 409.085,119.29 410.126,120.522 411.167,121.752 412.207,122.981 413.248,124.208 414.289,125.436 415.33,126.664 416.371,127.893 417.411,129.121 
 418.452,130.35 419.493,131.577 420.534,132.804 421.574,134.03 422.615,135.256 423.656,136.481 424.697,137.705 425.738,138.929 426.778,140.153 427.819,141.376 
 428.86,142.599 429.901,143.822 430.941,145.044 431.982,146.266 433.023,147.487 434.064,148.707 435.105,149.927 436.145,151.147 437.186,152.366 438.227,153.585 
 439.268,154.803 440.308,156.021 441.349,157.238 442.39,158.455 443.431,159.672 444.472,160.888 445.512,162.103 446.553,163.318 447.594,164.533 448.635,165.747 
 449.675,166.96 450.716,168.173 451.757,169.386 452.798,170.598 453.839,171.81 454.879,173.021 455.92,174.232 456.961,175.443 458.002,176.652 459.042,177.861 
 460.083,179.069 461.124,180.277 462.165,181.484 463.206,182.692 464.246,183.9 465.287,185.108 466.328,186.315 467.369,187.52 468.409,188.725 469.45,189.927 
 470.491,191.128 471.532,192.329 472.573,193.53 473.613,194.732 474.654,195.937 475.695,197.142 476.736,198.348 477.776,199.552 478.817,200.753 479.858,201.949 
 480.899,203.14 481.94,204.328 482.98,205.516 484.021,206.706 485.062,207.899 486.103,209.097 487.143,210.3 488.184,211.505 489.225,212.712 490.266,213.921 
 
 "/>
<polyline clip-path="url(#clip3102)" style="stroke:#e26f46; stroke-width:3; stroke-opacity:1; fill:none" points="
 256.221,124.207 256.968,125.095 257.715,125.985 258.461,126.877 259.208,127.773 259.955,128.675 260.702,129.583 261.448,130.496 262.195,131.412 262.942,132.329 
 263.688,133.242 264.435,134.152 265.182,135.056 265.928,135.957 266.675,136.856 267.422,137.755 268.169,138.657 268.915,139.561 269.662,140.466 270.409,141.373 
 271.155,142.28 271.902,143.185 272.649,144.089 273.396,144.992 274.142,145.894 274.889,146.795 275.636,147.697 276.382,148.599 277.129,149.502 277.876,150.404 
 278.623,151.307 279.369,152.209 280.116,153.111 280.863,154.012 281.609,154.914 282.356,155.814 283.103,156.715 283.849,157.616 284.596,158.516 285.343,159.417 
 286.09,160.317 286.836,161.217 287.583,162.117 288.33,163.016 289.076,163.916 289.823,164.815 290.57,165.714 291.317,166.613 292.063,167.511 292.81,168.41 
 293.557,169.308 294.303,170.206 295.05,171.104 295.797,172.001 296.544,172.898 297.29,173.796 298.037,174.693 298.784,175.589 299.53,176.486 300.277,177.382 
 301.024,178.278 301.77,179.174 302.517,180.069 303.264,180.965 304.011,181.86 304.757,182.755 305.504,183.65 306.251,184.545 306.997,185.439 307.744,186.332 
 308.491,187.226 309.238,188.119 309.984,189.012 310.731,189.905 311.478,190.799 312.224,191.692 312.971,192.585 313.718,193.478 314.465,194.369 315.211,195.259 
 315.958,196.148 316.705,197.038 317.451,197.927 318.198,198.818 318.945,199.71 319.691,200.603 320.438,201.496 321.185,202.387 321.932,203.277 322.678,204.163 
 323.425,205.046 324.172,205.926 324.918,206.807 325.665,207.689 326.412,208.573 327.159,209.462 327.905,210.354 328.652,211.249 329.399,212.145 330.145,213.042 
 
 "/>
<polyline clip-path="url(#clip3102)" style="stroke:#3da44d; stroke-width:3; stroke-opacity:1; fill:none" points="
 177.139,167.797 177.708,168.48 178.277,169.164 178.846,169.851 179.415,170.54 179.985,171.235 180.554,171.934 181.123,172.638 181.692,173.344 182.261,174.05 


We find that more durable capital leads to higher steady state investment and slows the rate of convergence for capital (the slopes are roughly the same, which implies that relative to steady state capital investment responds stronger at higher $\delta$, this in addition to the direct effect of depreciation).

# Use the model to simulate

We will use the deterministic steady-state as a starting point.

In [48]:
set_calibration!(model, delta=0.05)
# s0 = model.calibration[:states]
# [zip(model.symbols[:states], s0)...]

Dolo.ModelCalibration(Dolo.FlatCalibration(:alpha=>0.33,:n=>0.33,:beta=>0.99,:eta=>1.0,:sigma=>5.0,:delta=>0.05,:rho=>0.8,:zbar=>0.0,:z=>0.0,:sig_z=>0.016…), Dolo.GroupedCalibration(:values=>[-59.0785],:controls=>[0.33, 0.20961],:states=>[0.0, 4.19221],:exogenous=>[0.0],:rewards=>[-7.56209],:expectations=>[19.1811],:parameters=>[0.99, 5.0, 1.0, 90.1014, 0.05, 0.33, 0.8, 0.0, 0.016]), Dict(:alpha=>(:parameters, 6),:n=>(:controls, 1),:rho=>(:parameters, 7),:V=>(:values, 1),:delta=>(:parameters, 5),:eta=>(:parameters, 3),:sigma=>(:parameters, 2),:k=>(:states, 2),:sig_z=>(:parameters, 9),:z=>(:states, 1)…), DataStructures.OrderedDict(:values=>Symbol[:V],:controls=>Symbol[:n, :i],:states=>Symbol[:z, :k],:exogenous=>Symbol[:e_z],:rewards=>Symbol[:u],:expectations=>Symbol[:m],:parameters=>Symbol[:beta, :sigma, :eta, :chi, :delta, :alpha, :rho, :zbar, :sig_z]))

## Impulse response functions

Let us plot the response of consumption and investment to a shock on productivity (to innovation `e_z`)

In [49]:
irf = response(model, dr_global, :e_z, 0.01)

2-dimensional AxisArray{Float64,2,...} with axes:
    :V, Symbol[:e_z, :z, :k, :n, :i, :w, :rk, :y, :c]
    :T, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  31, 32, 33, 34, 35, 36, 37, 38, 39, 40]
And data, a 9×40 Array{Float64,2}:
 0.0       0.01       0.0        …  0.0         0.0         0.0       
 0.0       0.01       0.008         3.24519e-6  2.59615e-6  2.07692e-6
 4.19221   4.19226    4.20071       4.19919     4.19881     4.19846   
 0.330025  0.330942   0.330495      0.329807    0.329819    0.32983   
 0.20966   0.218072   0.216292      0.209586    0.209589    0.209593  
 1.5501    1.56425    1.56286    …  1.5513      1.55123     1.55117   
 0.060104  0.0608206  0.0605623     0.0600108   0.0600157   0.0600204 
 0.763541  0.772652   0.770924      0.763625    0.76362     0.763616  
 0.553881  0.554581   0.554632      0.554039    0.554031    0.554023  

The easiest way to plot IRF is simply using the stored `irf` values.

In [50]:
figure_3 = plot(
    plot(irf[:z], title = "Productivity", line = 2, legend=:none),
    plot(irf[:i], title = "Investment", line = 2, legend=:none),
    plot(irf[:n], title = "Labour", line = 2, legend=:none),
    plot(irf[:c], title = "Consumption", line =2, legend=:none)
)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 20 
 
 
 30 
 
 
 40 
 
 
 0.000 
 
 
 0.002 
 
 
 0.004 
 
 
 0.006 
 
 
 0.008 
 
 
 0.010 
 
 
 Productivity 
 
<polyline clip-path="url(#clip3302)" style="stroke:#009af9; stroke-width:2; stroke-opacity:1; fill:none" points="
 59.2368,175.869 64.9913,31.4961 70.7457,60.3706 76.5002,83.4702 82.2546,101.95 88.0091,116.734 93.7635,128.561 99.518,138.022 105.272,145.592 111.027,151.647 
 116.781,156.491 122.536,160.367 128.29,163.467 134.045,165.947 139.799,167.932 145.554,169.519 151.308,170.789 157.062,171.805 162.817,172.618 168.571,173.268 
 174.326,173.788 180.08,174.204 185.835,174.537 191.589,174.803 197.344,175.016 203.098,175.187 208.852,175.323 214.607,175.432 220.361,175.52 226.116,175.589 
 231.87,175.645 237.625,175.69 243.379,175.726 249.134,175.754 254.888,175.777 260.642,175.795 266.397,175.81 272.151,175.822 277.906,175.831 283.66,175.839 
 
 "/>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 20 
 
 
 30 
 
 
 40 
 
 
 0.210 
 
 
 0.212 
 
 
 0.214 
 
 
 0.216 
 
 
 0.218 
 
 
 Investment 
 
<polyline clip-path="url(#clip3303)" style="stroke:#009af9; stroke-width:2; stroke-opacity:1; fill:none" points="
 349.201,173.985 355.127,31.4961 361.053,61.6414 366.979,85.6412 372.905,104.737 378.831,119.921 384.757,131.982 390.683,141.553 396.609,149.137 402.535,155.138 
 408.461,159.876 414.387,163.608 420.313,166.54 426.239,168.834 432.165,170.622 438.091,172.008 444.017,173.074 449.943,173.888 455.869,174.503 461.795,174.96 
 467.721,175.293 473.647,175.529 479.573,175.69 485.499,175.791 491.425,175.847 497.351,175.869 503.277,175.863 509.203,175.839 515.129,175.799 521.055,175.749 
 526.981,175.692 532.907,175.63 538.833,175.565 544.759,175.499 550.685,175.433 556.611,175.367 562.537,175.303 568.463,175.24 574.389,175.18 580.315,175.122 
 
 "/>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 20 
 
 
 30 
 
 
 40 
 
 
 0.32950 
 
 
 0.32975 
 
 
 0.33000 
 
 
 0.33025 
 
 
 0.33050 
 
 
 0.33075 
 
 
 Labour 
 
<polyline clip-path="url(#clip3304)" style="stroke:#009af9; stroke-width:2; stroke-opacity:1; fill:none" points="
 59.2368,312.662 64.9913,231.496 70.7457,271.03 76.5002,301.248 82.2546,324.107 88.0091,341.164 93.7635,353.659 99.518,362.574 105.272,368.693 111.027,372.637 
 116.781,374.899 122.536,375.869 128.29,375.856 134.045,375.106 139.799,373.81 145.554,372.12 151.308,370.153 157.062,368.004 162.817,365.743 168.571,363.425 
 174.326,361.093 180.08,358.778 185.835,356.505 191.589,354.291 197.344,352.147 203.098,350.083 208.852,348.104 214.607,346.213 220.361,344.41 226.116,342.697 
 231.87,341.071 237.625,339.53 243.379,338.073 249.134,336.696 254.888,335.396 260.642,334.17 266.397,333.013 272.151,331.924 277.906,330.899 283.66,329.933 
 
 "/>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 20 
 
 
 30 
 
 
 40 
 
 
 0.5540 
 
 
 0.5542 
 
 
 0.5544 
 
 
 0.5546 
 
 
 Consumption 
 
<polyline clip-path="url(#clip3305)" style="stroke:#009af9; stroke-width:2; stroke-opacity:1; fill:none" points="
 355.892,375.869 361.646,248.523 367.4,239.217 373.155,233.859 378.909,231.528 384.664,231.496 390.418,233.19 396.173,236.16 401.927,240.052 407.682,244.588 
 413.436,249.554 419.19,254.781 424.945,260.143 430.699,265.539 436.454,270.896 442.208,276.159 447.963,281.286 453.717,286.25 459.472,291.03 465.226,295.614 
 470.98,299.995 476.735,304.17 482.489,308.14 488.244,311.906 493.998,315.475 499.753,318.851 505.507,322.042 511.262,325.054 517.016,327.896 522.77,330.574 
 528.525,333.098 534.279,335.475 540.034,337.712 545.788,339.816 551.543,341.796 557.297,343.658 563.052,345.409 568.806,347.055 574.561,348.601 580.315,350.055 
 
 "/>

The result of irf is The result is a superconvenient AxisArrays object. In case there is any disagreement about the meanging of signifier "superconvenient", the result can be converted to a DataFrame with the following function:

In [51]:
#using AxisArrays
#using DataFrames
#function to_DataFrame(x::AxisArray{Float64,2})
#    _axes = Dict(zip(AxisArrays.axisnames(irf), AxisArrays.axisvalues(irf)))
#    colnames = _axes[:V]
#    linenames = _axes[:T]
#    dd = Dict(v=>irf[Axis{:V}(v)].data for v in colnames)
#    return DataFrame(dd)
#end
#irf_df = to_DataFrame(irf)

## Stochastic simulations

Now we run 1000 random simulations the result is an AxisArrays indexed by:
- $T$ the number of dates
- $N$ the number of simulations
- $V$ is the number of variables
(the actual ordering of the dimensions is irrelevent if one uses the AxisArrays indexing routines)

In [68]:
sim = simulate(model, dr_global, N=1000, T=500)

# actual ordering of the data: (N,V,T)
size(sim.data)

(1000, 9, 500)

We plot the responses of consumption, investment and labour to the stochastic path of productivity.

In [61]:
# May have trouble increasing the number plotted on OSX.
imin = 500
imax = 540

p1 = plot(title = "Productivity")
for i in imin:imax
    plot!(sim[Axis{:N}(i),Axis{:V}(:z)], color=:red, alpha=0.1, xlim = [-10, 1010], ylim = [-0.12, 0.12], legend=:none)
end

p2 = plot(title = "Investment")
for i in imin:imax
    plot!(sim[Axis{:N}(i),Axis{:V}(:i)], color=:red, alpha=0.1, xlim = [-10, 1010], ylim = [0.12, 0.32], legend=:none)
end

p3 = plot(title = "Labour")
for i in imin:imax
    plot!(sim[Axis{:N}(i),Axis{:V}(:n)], color=:red, alpha=0.1, xlim = [-10, 1010], ylim = [0.31, 0.355], legend=:none)
end

p4 = plot(title = "Consumption")
for i in imin:imax
    plot!(sim[Axis{:N}(i),Axis{:V}(:c)], color=:red, alpha=0.1, xlim = [-10, 1010], ylim = [0.53, 0.575], legend=:none)
end

figure_4 = plot(p1, p2, p3, p4)


<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 250 
 
 
 500 
 
 
 750 
 
 
 1000 
 
 
 -0.10 
 
 
 -0.05 
 
 
 0.00 
 
 
 0.05 
 
 
 0.10 
 
 
 Productivity 
 
<polyline clip-path="url(#clip3702)" style="stroke:#ff0000; stroke-width:1; stroke-opacity:0.1; fill:none" points="
 48.7553,103.682 48.9883,101.825 49.2213,112.097 49.4543,116.391 49.6872,115.831 49.9202,103.792 50.1532,122.945 50.3862,114.93 50.6192,117.311 50.8521,134.624 
 51.0851,137.098 51.3181,136.903 51.5511,129.592 51.784,112.875 52.017,106.883 52.25,108.779 52.483,124.624 52.7159,125.856 52.9489,130.708 53.1819,115.189 
 53.4149,110.86 53.6479,98.3968 53.8808,87.8815 54.1138,90.579 54.3468,66.609 54.5798,87.1654 54.8127,83.6841 55.0457,70.7004 55.2787,62.6003 55.5117,86.5812 
 55.7446,75.5285 55.9776,72.5134 56.2106,80.2725 56.4436,82.1259 56.6765,91.9424 56.9095,107.511 57.1425,109.389 57.3755,94.8773 57.6085,92.1977 57.8414,94.6496 
 58.0744,98.9768 58.3074,86.5376 58.5404,94.4755 58.7733,114.801 59.0063,104.268 59.2393,106.324 59.4723,107.399 59.7052,116.737 59.9382,116.247 60.1712,105.722 
 60.4042,105.075 60.6371,95.6033 60.8701,88.9274 61.1031,94.7305 61.3361,95.9429 61.5691,114.491 61.802,104.124 62.035,113.366 62.268,126.255 62.501,128.556 
 62.7339,131.818 62.9669,126.983 63.1999,112.669 63.4329,83.6387 63.6658,94.4579 63.8988,90.3065 64.1318,94.3212 64.3648,95.0777 64.5978,110.116 64.8307,103.881 
 65.0637,117.236 65.2967,119.586 65.5297,123.12 65.7626,108.094 65.9956,105.014 66.2286,110.508 66.4616,106.121 66.6945,106.305 66.9275,103.422 67.1605,103.202 
 67.3935,94.7529 67.6264,86.1551 67.8594,111.264 68.0924,106.337 68.3254,114.03 68.5584,139.243 68.7913,145.216 69.0243,155.606 69.2573,130.588 69.4903,119.895 
 69.7232,108.236 69.9562,101.56 70.1892,84.2269 70.4222,86.7064 70.6551,90.7437 70.8881,85.6572 71.1211,107.39 71.3541,120.169 71.587,111.339 71.82,100.273 
 72.053,90.3398 72.286,79.2274 72.519,86.2451 72.7519,104.435 72.9849,107.02 73.2179,107.797 73.4509,100.361 73.6838,100.96 73.9168,98.0867 74.1498,97.8345 
 74.3828,97.7698 74.6157,105.588 74.8487,110.673 75.0817,100.751 75.3147,93.7634 75.5477,96.9372 75.7806,87.7153 76.0136,83.0646 76.2466,74.8694 76.4796,67.0263 
 76.7125,73.9205 76.9455,84.1117 77.1785,95.6926 77.4115,86.7073 77.6444,86.8916 77.8774,78.6726 78.1104,85.7407 78.3434,85.8641 78.5763,77.4748 78.8093,68.5637 
 79.0423,80.9498 79.2753,73.3585 79.5083,74.0561 79.7412,79.5715 79.9742,83.2901 80.2072,97.6604 80.4402,86.8541 80.6731,82.5554 80.9061,99.1625 81.1391,116.948 
 81.3721,117.584 81.605,120.689 81.838,124.762 82.071,124.336 82.304,102.727 82.5369,115.028 82.7699,110.704 83.0029,97.1043 83.2359,102.578 83.4689,90.3519 
 83.7018,71.4665 83.9348,66.5085 84.1678,88.8402 84.4008,84.8897 84.6337,88.9094 84.8667,76.9637 85.0997,79.3501 85.3327,81.6826 85.5656,71.9711 85.7986,73.9633 
 86.0316,77.114 86.2646,80.4383 86.4975,84.8828 86.7305,91.7895 86.9635,78.5956 87.1965,95.9619 87.4295,96.7244 87.6624,72.8396 87.8954,92.5583 88.1284,85.6234 
 88.3614,67.8479 88.5943,71.2282 88.8273,90.622 89.0603,88.9058 89.2933,98.3963 89.5262,95.9179 89.7592,107.833 89.9922,116.849 90.2252,109.72 90.4582,120.029 
 90.6911,133.019 90.9241,126.563 91.1571,114.688 91.3901,112.253 91.623,110.12 91.856,105.469 92.089,107.947 92.322,98.4425 92.5549,102.399 92.7879,109.082 
 93.0209,101.053 93.2539,109.368 93.4868,97.9896 93.7198,94.7721 93.9528,102.597 94.1858,94.87 94.4188,79.2427 94.6517,74.5792 94.8847,83.9267 95.1177,97.5937 
 95.3507,84.3794 95.5836,91.3969 95.8166,106.008 96.0496,96.351 96.2826,101.677 96.5155,101.895 96.7485,110.396 96.9815,120.365 97.2145,114.653 97.4474,109.864 
 97.6804,99.6323 97.9134,107.198 98.1464,110.856 98.3794,100.44 98.6123,90.6283 98.8453,108.506 99.0783,99.3269 99.3113,96.6925 99.5442,101.854 99.7772,106.651 
 100.01,100.491 100.243,83.7032 100.476,94.5888 100.709,111.822 100.942,105.467 101.175,101.583 101.408,131.103 101.6

It's easy to compare the simulated result with the deterministic steady state value implied from the theoretical model:

In [62]:
mean(sim[Axis{:N}(540),Axis{:V}(:n)])

0.33078987964453366

In [63]:
mean(sim[Axis{:N}(540),Axis{:V}(:i)])

0.20799914504231629

In [25]:
mean(sim[Axis{:N}(540),Axis{:V}(:k)])

5.513662837368409

In [26]:
#set_calibration!(model,delta=0.05)
model.calibration[:controls]

2-element Array{Float64,1}:
 0.33   
 0.20961

In [27]:
model.calibration[:states]

2-element Array{Float64,1}:
 0.0    
 4.19221